# DSCI 511: Data Acquisition and Pre-Processing - Final Project <br> 

# 3. Increasing the Granularity of Data

<font size="4">**3.1. Awards**<font>

In [2]:
from bs4 import BeautifulSoup
import urllib 
import requests
import re
from collections import defaultdict
from collections import Counter
import json

IMDB provides a detailed level of information about different types of awards, movie festivals or companies that these awards are sponsered by and whether a movie was only nominated for this award or won it as well. It is reasonable to summarize the frequency of pertinent data for each title, using counter object for the following data analysis objectives.

- Below script extracts these information for up to 100 movies at once and eventually write the data that we get into a JSON file called "awrds_clean".

- We will use `regex` and `Counter` to count the number of Awards from a particular Awards Company, and the Award Category.

In [3]:
    # 1. Line below opens the file.
file = open("awards_clean.json", "w")

     # 2. The IMDB website lists 100 movies per page.  Hence, if we change our range below, to be "range(1,10)"...
#...it would get information on 1,000 movies.
# Next, we put the data into Beautiful Soup format, stored in "soup".  Page1 link is below to see the website:..
#..link:  https://www.imdb.com/list/ls003495084/?sort=date_added,desc&st_dt=&mode=detail&page=1  
i = 1
for item in range(1,2):
    url = 'https://www.imdb.com/list/ls003495084/?sort=date_added,desc&st_dt=&mode=detail&page=' + str(item)
    html_text = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html_text, 'html.parser')

    # 3. Of the 100 movies on page 1, we grab every individual movie link. 
    movies={}
    for x in soup.find_all('a'):
        if x.get("href") is not None:
            if re.search('^/title/tt', x['href']) and x.text!=' \n':
                movies[x.text]={'link':'https://www.imdb.com/'+x['href']}     
    
    # ***Side note before we go to Step 4:  The link for each movie is exactly the same.  The only...
#...difference in the links is a large number.  
    
    # 4.  Based on side note above, we decide to pull out these numbers in each link and put it into a..
#...list called movie_numbers 
    movie_numbers = []
    for key, dictionary in movies.items():
        for values in dictionary.values():
            movie_number = re.search("[0-9]+", values)
            movie_numbers.append(movie_number.group(0))
    
    
    # 5. We can now iterate through movie_numbers (from step 4) to successfully reach in and grab the info...
#...for the 100 individual movies.  In fact, we want to do more than this.  We want to click on our movie...
#...link, but then we also want to click on the Awards link that is within that movie link.  Well, every...
#...Awards link of each movie is also the same (besides for the movie number).  Therefore, we can iterate...
#...through each movies inner awards link.
# We will store the info from the awards page for each movie into a Beautiful Soup object called "movie".
    Awards = {}
    for number in movie_numbers:
        def get_url(M):
            return urllib.request.urlopen("https://www.imdb.com/title/tt" + M + "/awards?ref_=tt_ql_op_1").read()
        html_text = get_url(number)    
        movie = BeautifulSoup(html_text, 'html.parser')
        
    # 6. The Beautiful Soup functions below creates two dictionaries.  One contains a list of the awards the..
#.. movie was "Nominated" for, and the other holds the awards the movie "Won."  Each dictionaries values
#... lists the Company that gave the Award, the Award category, and names the individuals who won individual..
#... awards. 
        tds = movie.find_all("td")
        d = defaultdict(list)
        Win_Nom = ''
        try:
            for td in tds:
                split_line = (td.text.split("\n"))
                for line in split_line:
                    if line == "Nominee":
                        Win_Nom = 'Nominee'
                    elif line == "Winner":
                        Win_Nom = "Winner"
                    elif Win_Nom == 'Nominee':
                        piece = (line.split("  "))
                        if piece[0] != '': 
                            d["Nominee"].append(line)    
                        if len(piece) > 5:
                            piece[6].strip()
                            d["Nominee"].append(piece[6])
                    elif Win_Nom == "Winner":
                        piece = line.split("  ")
                        if piece[0] != '':
                            d["Winner"].append(line)
                        if len(piece) > 1:
                            piece[6].strip()
                            d["Winner"].append(piece[6])
    
    # 7.  The below code takes the two dictionaries from step 6, and creates a Counter structure which is...
#... what we are going to dump into a JSON file.  However, it is possible that Step 6's dictionary structure..
#... would be preferable for an analysis. 
            num_awards = Counter() 
            for keys, values in d.items():
                for value in values:
                    award_company = re.search("(.* Award)", value)
                    award_category = re.search("(Best .*)", value)
                    if award_company != None:
                        num_awards[award_company.group(0)] = num_awards[award_company.group(0)] + 1
                    if award_category != None:
                        num_awards[award_category.group(0)] = num_awards[award_category.group(0)] + 1
            Awards.update({str(i):num_awards})
            i+=1
                 
    # 8.  Now, not every movie was able to be organized with the code above.  If so, the code below ran.
# Everything is the same below, except for a couple of minor tweaks.  The only difference in the output...
# is that there is extra white space in these dictionaries.  But, the Counter structure eliminates this...
# extra white space !
        except IndexError:
            for td in tds:
                split_line = (td.text.split("\n"))
                for line in split_line:
                    if line == "Nominee":
                        Win_Nom = 'Nominee'
                    elif line == "Winner":
                        Win_Nom = "Winner"
                    elif Win_Nom == 'Nominee':
                        piece = (line.split("  "))
                        d["Nominee"].append(line)    
                    elif Win_Nom == "Winner":
                        piece = line.split("  ")
                        d["Winner"].append(line)      
    # 9.  The counter structure.
            num_awards = Counter() 
            for keys, values in d.items():
                for value in values:
                    award_company = re.search("(.* Award)", value)
                    award_category = re.search("(Best .*)", value)
                    if award_company != None:
                        num_awards[award_company.group(0)] = num_awards[award_company.group(0)] + 1
                    if award_category != None:
                        num_awards[award_category.group(0)] = num_awards[award_category.group(0)] + 1
            Awards.update({str(i):num_awards})
            i+=1
            
    # 10.  Finally, we can dump our Counter Structures for the 100 movies into a JSON format. 
counter_ds = json.dumps(Awards)
file.write(counter_ds)

87572